# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [100]:
# initial imports
import os
import panel as pn
pn.extension()
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import panel as pn
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv

# Initialize the Panel Extensions (for Plotly)

In [2]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX")

# Import Data

In [3]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")  #making path to census data
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")      #making path to coordinate data
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [93]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    # YOUR CODE HERE!
    sfo_data_grp=sfo_data["housing_units"].groupby(sfo_data.index).mean() #refining the dataframe
    ax=sfo_data_grp.hvplot(kind='bar',title="Housing Units in San Francisco from 2010 to 2016",
    rot=90).opts(yformatter="%.0f",ylabel="Housing Units",ylim=(min(sfo_data_grp),max(sfo_data_grp)),hover_color="green",nonselection_alpha=.4)
    #ax=px.bar(sfo_data_grp,title='Housing Units in San Francisco from 2010 to 2016')
    #ax.set_xlabel("Year")
    #ax.set_ylabel("Housing Units")
    #ax.set_title('Housing Units in San Francisco from 2010 to 2016',fontweight="bold")
    return(ax)
                                
    
def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
    grp_avg=sfo_data[["sale_price_sqr_foot","gross_rent"]].groupby(sfo_data.index).mean() #collecting the data
    ax=grp_avg["gross_rent"].hvplot(kind='line',x='year',y='gross_rent',
    title="Average Gross Rent in San Francisco").opts(xlabel="Year",ylabel="Gross Rent",color="red")
    #ax.set_xlabel("Year")
    #ax.set_ylabel("Gross Rent")
    #ax.set_title("Average Gross Rent in San Francisco",fontweight="bold")
    return(ax)

def average_sales_price():
    """Average Sales Price Per Year."""
    # YOUR CODE HERE!
    
    grp_avg=sfo_data[["sale_price_sqr_foot","gross_rent"]].groupby(sfo_data.index).mean() #collecting the data
    ax=grp_avg.hvplot(kind="line",x='year',
    y="sale_price_sqr_foot").opts(xlabel="Year",ylabel="Avg Sale Price",title="Average Sale Price per Square Foot in San Francisco")
    #ax=px.bar(grp_avg,title="Average Sale Price per Square Foot in San Francisco")
    #ax.set_xlabel('Year')
    #ax.set_ylabel('Avg Sale Price')
    #ax.set_title("Average Sale Price per Square Foot in San Francisco",fontweight="bold")
    return(ax)

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    price_avg_year_neighbor=sfo_data[["neighborhood","sale_price_sqr_foot"]].groupby([sfo_data.index,"neighborhood"]).mean()
    ax=price_avg_year_neighbor.hvplot(kind='line',x='year',y='sale_price_sqr_foot',
    groupby='neighborhood',width=500).opts(xlabel="Year",ylabel="Avg Sale Price Per Square Foot")
    return(ax)      

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    # YOUR CODE HERE!
    neighborhood_grp=sfo_data.groupby(["neighborhood"]).mean().sort_values(by='sale_price_sqr_foot',ascending=False)
    ax=neighborhood_grp.head(10).hvplot(kind='bar'
    ,label="Top 10 Expensive Neighborhoods in SFO",x='neighborhood',y='sale_price_sqr_foot',rot=90,
    figsize=(30,10)).opts(ylabel="Avg Price Per Square Foot",ylim=(0,1000))
    return(ax)
                                
def parallel_coordinates():
    """Construts Parallel Coordinates Plot from San Francisco Housing Data."""
    
    # YOUR CODE HERE!
    neighborhood_grp=sfo_data.groupby(["neighborhood"]).mean().sort_values(by='sale_price_sqr_foot',ascending=False)
    ax=px.parallel_coordinates(neighborhood_grp.head(10),
    dimensions=['sale_price_sqr_foot','housing_units','gross_rent'],color='sale_price_sqr_foot'
    ,title="Parallel Coordinates Plot")
    return(ax)

def parallel_categories():
    """Constructs Parallel Categories Plot from San Francisco Housing Data."""
    
    # YOUR CODE HERE!
    neighborhood_grp=sfo_data.groupby(["neighborhood"]).mean().sort_values(by='sale_price_sqr_foot',ascending=False)
    ax=px.parallel_categories(
    neighborhood_grp.head(10).reset_index().copy(),
    dimensions=["neighborhood","sale_price_sqr_foot","housing_units", "gross_rent"],
    color='sale_price_sqr_foot',title="Parallel Categories Plot"
    )
    return(ax)


def neighborhood_map():
    """Constructs Neighborhood Map from San Francisco Housing Data"""
    
    # Calculate the mean values for each neighborhood
    neighborhood_grp=sfo_data.groupby(["neighborhood"]).mean().sort_values(by='sale_price_sqr_foot',ascending=False)
    neighborhood_grp.index.name="Neighborhood"  #Changing the name of the index
    
    # Join the average values with the neighborhood locations
    df_neighborhood=pd.concat([df_neighborhood_locations.set_index("Neighborhood").copy(),neighborhood_grp],axis='columns',join='inner')

    # Create a scatter mapbox to analyze neighborhood info
    px.set_mapbox_access_token(mapbox_token)
    ax=px.scatter_mapbox(df_neighborhood,lat="Lat",lon="Lon",color="gross_rent",zoom=3,width=1000,
    color_continuous_scale=px.colors.cyclical.IceFire,title="Averag Sale Price Per Square Foot and Gross Rent in San Francisco",
    size="sale_price_sqr_foot")
    return(ax)

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [98]:
# YOUR CODE HERE!


# Tab for Bar Plots

bar_column = pn.Column(
    "## Bar Graphs of San Francisco Housing Data", 
    housing_units_per_year(),top_most_expensive_neighborhoods(),
)



#Tab for Line Graphs
line_column = pn.Column(
    "## Line Graphs of San Francisco Housing Data",
     average_price_by_neighborhood(),
    average_gross_rent(),
    average_sales_price(),
)

# Tab for Parallel Plots
parallel_column = pn.Column(
    "## Parallel Plots of San Francisco Housing Data",
    parallel_coordinates(),
    parallel_categories(),
)

# Tab for Geo Plot
geo_column = pn.Row(
    "## Neighborhood Map of San Francisco Housing Data",
    neighborhood_map(),
)

# Create tabs
SanFran_dashboard = pn.Tabs(
    ("Bar Plots", bar_column), 
    ("Line Plots", line_column), 
    ("Parallel Plots",parallel_column),
    ("Neighborhood Plot",geo_column),
)



## Serve the Panel Dashboard

In [99]:
SanFran_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Bars)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [2] HoloViews(Curve)
        [3] HoloViews(Curve)
    [2] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [3] Row
        [0] Markdown(str)
        [1] Plotly(Figure)